# Enriching CMU dataset
## IMDb_ID to CMU
We will first fetch IMDb_ID to CMU on matching FreebaseID from Wikidata using sparql

In [1]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
import sys
import requests
import numpy as np

sys.path.append('../../data')
sys.path.append('../../pickles')

# Load your CSV file with movie data
df = pd.read_pickle('../../pickles/cmu_movies_df.pkl')

In [2]:
def fetch_wikidata_request(offset):
  url = 'https://query.wikidata.org/sparql'
  query = f"""
    SELECT ?item ?IMDb_ID ?FreebaseID WHERE {{
      ?item wdt:P31 wd:Q11424 .
      ?item wdt:P646 ?FreebaseID .
      ?item wdt:P345 ?IMDb_ID .
    }} 
    ORDER BY ?IMDb_ID
    OFFSET {offset}
    """
  r = requests.get(url, params = {'format': 'json', 'query': query})
  data = r.json()
  # print(data)
  return data

def process_data(data):
  """
  Extracts a dataframe with FreebaseID and IMDb_ID from the JSON data returned by the query
  Returns: the DataFrame with FreebaseID and IMDb_ID
  """
  transformed_data = [
      {
          'Freebase Movie ID': entry.get('FreebaseID', {}).get('value', None),
          'IMDb_ID': entry.get('IMDb_ID', {}).get('value', None)
      }
      for entry in data['results']['bindings']
  ]

  # Create a DataFrame
  df = pd.DataFrame(transformed_data)
  return df

In [3]:
num_calls = 1 # no need for more, because the call returns all
offset = 0
for i in range(num_calls):
  data = fetch_wikidata_request(offset)
  new_df = process_data(data)
  if len(new_df) > 0:
    df = pd.merge(df, new_df, how='left', on='Freebase Movie ID')
  offset += len(new_df)
  print('offset', offset)
  

offset 106112


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81847 entries, 0 to 81846
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Wikipedia Movie ID    81847 non-null  int64  
 1   Freebase Movie ID     81847 non-null  object 
 2   Movie Name            81847 non-null  object 
 3   Release Date          74927 non-null  object 
 4   Box Office Revenue    8403 non-null   float64
 5   Runtime               61352 non-null  float64
 6   Language Freebase ID  81847 non-null  object 
 7   Language Name         81847 non-null  object 
 8   Country Freebase ID   81847 non-null  object 
 9   Country Name          81847 non-null  object 
 10  Genre Freebase ID     81847 non-null  object 
 11  Genre Name            81847 non-null  object 
 12  IMDb_ID               63342 non-null  object 
dtypes: float64(2), int64(1), object(10)
memory usage: 8.1+ MB


In [5]:
# # Uncomment if you want to store the data without TMDB
# df.to_pickle('../../pickles/cmu_movies_with_imdb_id.pkl')

## TMDB to CMU

Once we have IMDb_ID in the CMU dataset, we can easily match the tmdb 1 Million dataset (or any other tmdb dataset) on IMDb_ID

In [6]:
def tmdb1m_to_pickle():
  """
  Use initially, to extract the data into a pickle data frame
  """

  tmdb_df = pd.read_csv('data/tmdb/TMDB_all_movies.csv')
  tmdb_df.to_pickle('pickles/tmdb1m')

# tmdb1m_to_pickle() # use initially to extract data from csv to pickle

In [7]:
# load tmdb1m
tmdb = pd.read_pickle('../../pickles/tmdb1m')
tmdb.rename(columns={'imdb_id': 'IMDb_ID'}, inplace=True)

In [8]:
tmdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1016527 entries, 0 to 1016526
Data columns (total 28 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   id                       1016527 non-null  int64  
 1   title                    1016516 non-null  object 
 2   vote_average             1016525 non-null  float64
 3   vote_count               1016525 non-null  float64
 4   status                   1016525 non-null  object 
 5   release_date             903854 non-null   object 
 6   revenue                  1016525 non-null  float64
 7   runtime                  1016525 non-null  float64
 8   budget                   1016525 non-null  float64
 9   IMDb_ID                  590237 non-null   object 
 10  original_language        1016525 non-null  object 
 11  original_title           1016516 non-null  object 
 12  overview                 837498 non-null   object 
 13  popularity               1016525 non-null 

In [9]:
tmdb.columns

# drop the rows where imdb_id is na from tmdb
tmdb.dropna(subset=['IMDb_ID'], inplace=True)

# select the COLUMNS we want to keep
tmdb = tmdb[['vote_average', 'vote_count', 'revenue', 'budget', 'IMDb_ID', 'popularity', 'production_companies', 'director', 'writers', 'producers', 'imdb_rating', 'imdb_votes']]

tmdb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 590237 entries, 0 to 1016524
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   vote_average          590237 non-null  float64
 1   vote_count            590237 non-null  float64
 2   revenue               590237 non-null  float64
 3   budget                590237 non-null  float64
 4   IMDb_ID               590237 non-null  object 
 5   popularity            590237 non-null  float64
 6   production_companies  345017 non-null  object 
 7   director              543103 non-null  object 
 8   writers               381627 non-null  object 
 9   producers             249461 non-null  object 
 10  imdb_rating           429072 non-null  float64
 11  imdb_votes            429072 non-null  float64
dtypes: float64(7), object(5)
memory usage: 58.5+ MB


In [10]:
df_merged = pd.merge(df, tmdb, how='left', on='IMDb_ID')

In [11]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81855 entries, 0 to 81854
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Wikipedia Movie ID    81855 non-null  int64  
 1   Freebase Movie ID     81855 non-null  object 
 2   Movie Name            81855 non-null  object 
 3   Release Date          74933 non-null  object 
 4   Box Office Revenue    8403 non-null   float64
 5   Runtime               61357 non-null  float64
 6   Language Freebase ID  81855 non-null  object 
 7   Language Name         81855 non-null  object 
 8   Country Freebase ID   81855 non-null  object 
 9   Country Name          81855 non-null  object 
 10  Genre Freebase ID     81855 non-null  object 
 11  Genre Name            81855 non-null  object 
 12  IMDb_ID               63350 non-null  object 
 13  vote_average          59403 non-null  float64
 14  vote_count            59403 non-null  float64
 15  revenue            

In [12]:
df_merged.to_pickle('../../pickles/cmu_imdb_tmdb_merged.pkl')